In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

def load_and_chunk_documents(file_path: str) -> List:
    """
    Загружает документ из файла и разбивает его на чанки.
    """
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1600,
        chunk_overlap=200,
        separators=[
            "\n# ", "\n## ", "\n- ", "\n\n", "\n", " ", ""
        ]
    )
    chunked_docs = text_splitter.split_documents(documents)
    print(f"Документ '{file_path}' разбит на {len(chunked_docs)} чанков.")
    return chunked_docs

In [2]:
chunked_docs = load_and_chunk_documents("data/docs.md")

Документ 'data/docs.md' разбит на 10 чанков.


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from typing import List
from langchain_core.documents import Document
import os
import shutil
from dotenv import load_dotenv

def create_vector_store(chunked_docs: List, collection_name: str = "credit_platform_docs"):
    """
    Создает векторное хранилище из чанков документов.
    """
    persist_dir = "chroma_db"
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)

    # Загрузка переменных окружения из .env
    load_dotenv()

    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )

    vector_store = Chroma.from_documents(
        documents=chunked_docs,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_dir
    )

    print(f"Векторное хранилище '{collection_name}' создано и содержит {len(vector_store.get()['ids'])} документов.")
    return vector_store

In [4]:
vector_store = create_vector_store(chunked_docs, "p2p_agent_db")

Векторное хранилище 'p2p_agent_db' создано и содержит 10 документов.


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma

RAG_PROMPT_TEMPLATE = """
Ты — дружелюбный ассистент, отвечающий по вопросам взаимного кредитования. 
Используй предоставленный ниже контекст и историю диалога, чтобы ответить на вопрос пользователя. Ответь коротко и по существу, дружелюбным тоном. 
Не пересказывай весь документ, а приведи только ту информацию, что отвечает на вопрос пользователя. 
Не добавляй информацию вне предоставленного контекста.
Не включай личные мнения, только факты
Не повторяй вопрос и не перечисляй пункты, если об этом не просили. 

История диалога:
{chat_history}

Контекст:
{context}

Вопрос:
{question}

Ответ:
"""

def format_docs(docs):
    """
    Форматирует список документов в единую строку.
    """
    return "\n\n".join(doc.page_content for doc in docs)

def extract_question(query):
    return query['q']

def extract_history(query):
    return query['h']

def create_qa_rag_chain(vector_store: Chroma):
    # Загрузка переменных окружения из .env
    load_dotenv()
    openai_api_key = os.getenv("OPENAI_API_KEY")

    retriever = vector_store.as_retriever(search_kwargs={"k": 2})

    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.2,
        openai_api_key=openai_api_key
    )
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

    rag_chain = ({
        "context": extract_question | retriever | format_docs, 
        "question": extract_question,
        "chat_history": extract_history
    }
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [19]:
p2p_agent_executor = create_qa_rag_chain(vector_store)

In [7]:
print("\n--- Тест 2 ---")
response2 = p2p_agent_executor.invoke(" При каких условиях это происходит?")
print("Ответ интеллектуального агента:", response2)


--- Тест 2 ---
Ответ интеллектуального агента: Ликвидация залога происходит, когда стоимость заблокированного ETH опускается ниже поддерживаемого уровня.


In [21]:
print("\n--- Тест 1 ---")
response1 = p2p_agent_executor.invoke({
    "q": "При каких условиях это происходит?",
    "h": "Пользователь: Что такое liquidation в peer-to-pool системе займов?\nАссистент: Liquidation в peer-to-pool системе займов происходит, когда заемщик не возвращает займ вовремя или стоимость залога (например, ETH) сильно упадет."
})
print("Ответ интеллектуального агента:", response1)


--- Тест 1 ---
Ответ интеллектуального агента: Liquidation в peer-to-pool системе займов происходит, когда заемщик не возвращает займ вовремя или стоимость залога (например, ETH) сильно упадет.
